<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# FBT Single Node on MovieLens (Python, CPU)

Lets say you are shopping online and you'd like to buy a Microsoft Surface tablet. You might add this to your shopping cart. You may then see on your screen a phrase similar to "Frequently bought together" with visuals and links to a Microsoft keyboard, a tablet case, a mouse and so on. Many recommendation algorithms can enable this feature under the hood. However, one of the simplest ones is exactly as the phrase describes - What are some other items that people bought along with a Microsoft surface? Which of these items are the most popular? That is essentially what is implemented in the FBT (Frequently bought together) recommender class which we work with in this notebook.

FBT recommender can be thought of a simple restriction of the SAR (Simple Algorithm for Recommendation) recommender. Like SAR, FBT is a fast and scalable algorithm for personalized recommendations based on user transaction history. SAR leverages user ratings of items and timestamp information of when user rated an item to produce easily explainable and interpretable recommendations. However, there are many scenarios where we may not have reliable rating information or timestamps. All we have is user interactions with items and we need a simple recommendation engine that can leverage this interaction information without regard to context or quality of interaction or when in history did this interaction happen.

This is where we can leverage FBT. Like SAR, FBT recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users that interacted with one item are also likely to have interacted with the other. Unlike SAR though, user ***affinity*** to an item is simply binary - 1 if the user has interacted with an item in the past, 0 otherwise. We don't associate quality of this interaction for this model that rating information can typically do for us.

### Advantages of FBT:
- A simple first algorithm to implement when all you have is users and items and no more information. Covers a broad range of customer scenarios.
- High accuracy for an easy to train and deploy algorithm
- Fast training and scoring, only requiring simple counting to construct matrices used at prediction time.
- Easily scalable to implement in Spark for large tables of user-item interactions.

### Notes to use FBT properly:
- Since FBT uses very little information, recommendations will likely not have more context than historical interactions. If we can leverage useful information from item or user features, more sohisticated algorithms will have an edge in performance.

- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- FBT does not need ratings information, hence we can't predict ratings either. Evaluation can best happen with user studies. We can still look at offline evaluation methods like Precision@K, Recall@K.

This notebook provides an example of how to utilize and evaluate FBT in Python on a CPU.

In [1]:
%reload_ext autoreload
%autoreload 2

import logging
import numpy as np
import pandas as pd
import scrapbook as sb

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)
from recommenders.models.fbt.fbt import FBT
print("Pandas version: {}".format(pd.__version__))

Pandas version: 1.1.5


In [2]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 1.1 Download and use the MovieLens Dataset

In [3]:
col_user = 'user_id'
col_item = 'item_id'
col_item_name = f'{col_item}_name'
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=(col_user, col_item),
    title_col=col_item_name
)
data.head()

100%|██████████| 4.81k/4.81k [00:01<00:00, 3.75kKB/s]


,user_id,item_id,item_id_name
0,196,242,Kolya (1996)
1,63,242,Kolya (1996)
2,226,242,Kolya (1996)
3,154,242,Kolya (1996)
4,306,242,Kolya (1996)


### 1.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because FBT generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.

In [4]:
train, test = python_stratified_split(data, 
                                      ratio=0.75, 
                                      col_user=col_user, 
                                      col_item=col_item, 
                                      seed=42)

In [5]:
print("""
Train:
Total Ratings: {train_total}
Unique Users: {train_users}
Unique Items: {train_items}

Test:
Total Ratings: {test_total}
Unique Users: {test_users}
Unique Items: {test_items}
""".format(
    train_total=len(train),
    train_users=len(train[col_user].unique()),
    train_items=len(train[col_item].unique()),
    test_total=len(test),
    test_users=len(test[col_user].unique()),
    test_items=len(test[col_item].unique()),
))


Train:
Total Ratings: 74992
Unique Users: 943
Unique Items: 1601

Test:
Total Ratings: 25008
Unique Users: 943
Unique Items: 1532



# 2 Train the FBT Model

In [6]:
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

col_score = 'score'
model = FBT(
    col_user=col_user,
    col_item=col_item,
    col_score=col_score
)

In [7]:
with Timer() as train_time:
    model.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

2021-09-14 14:40:40,547 INFO     Check input dataframe to fit() is of the type, schema we expect and there are no duplicates.
2021-09-14 14:40:40,563 INFO     Creating index columns
2021-09-14 14:40:40,655 INFO     Building user affinity sparse matrix
2021-09-14 14:40:40,713 INFO     Done training


Took 0.17020754999975907 seconds for training.


In [8]:
model.item_frequencies

array([ 86, 120,  52, ...,   1,   1,   1])

In [9]:
print(model.n_items, model.n_users)

1601 943


# 3. Make recommendations using the FBT model

In [15]:
# Predict 10 recommended items that user has not interacted with during training
with Timer() as test_time:
    topk_remove_seen = model.recommend_k_items(test=test, 
                                               top_k=10, 
                                               remove_seen=True)

print("Took {} seconds for prediction.".format(test_time.interval))

2021-09-14 14:45:18,154 INFO     Calculating recommendation scores
2021-09-14 14:45:18,343 INFO     Removing seen items


Took 0.216254740000295 seconds for prediction.


In [19]:
topk_remove_seen.head(10)

,user_id,item_id,score
0,1,98,13800.0
1,1,56,12657.0
2,1,69,12159.0
3,1,423,11940.0
4,1,204,11808.0
5,1,117,11528.0
6,1,288,10883.0
7,1,183,10776.0
8,1,238,10580.0
9,1,405,10428.0


In [17]:
# Predict 10 recommendations while retaining any items 
# user has already interacted with during training
with Timer() as test_time:
    topk_keep_seen = model.recommend_k_items(test=test, top_k=10, remove_seen=False)
print("Took {} seconds for prediction.".format(test_time.interval))

2021-09-14 14:46:23,063 INFO     Calculating recommendation scores


Took 0.2068116420014121 seconds for prediction.


In [18]:
topk_keep_seen.head(10)

,user_id,item_id,score
0,1,50,16763
1,1,181,15297
2,1,174,15081
3,1,1,14654
4,1,100,13913
5,1,98,13800
6,1,172,13776
7,1,210,13513
8,1,121,12923
9,1,222,12897


In [22]:
# Adding titles of recommended items for novel recommendations not seen during training
topk_remove_seen_with_titles = (
    topk_remove_seen.merge((
        data.loc[:, [col_item, col_item_name]]
            .drop_duplicates()
            .set_index(col_item)
    ), on=col_item, how='inner')
    .sort_values(by=[col_user, col_score], ascending=[True, False])
    .reset_index(drop=True)
)
        
topk_remove_seen_with_titles.head(10)

,user_id,item_id,score,item_id_name
0,1,98,13800.0,"Silence of the Lambs, The (1991)"
1,1,56,12657.0,Pulp Fiction (1994)
2,1,69,12159.0,Forrest Gump (1994)
3,1,423,11940.0,E.T. the Extra-Terrestrial (1982)
4,1,204,11808.0,Back to the Future (1985)
5,1,117,11528.0,"Rock, The (1996)"
6,1,288,10883.0,Scream (1996)
7,1,183,10776.0,Alien (1979)
8,1,238,10580.0,Raising Arizona (1987)
9,1,405,10428.0,Mission: Impossible (1996)


In [23]:
# Looking at top 10 titles for userid = 1
topk_keep_seen_with_titles = (
    topk_keep_seen.merge((
        data.loc[:, [col_item, col_item_name]]
            .drop_duplicates()
            .set_index(col_item)
    ), on=col_item, how='inner')
    .sort_values(by=[col_user, col_score], ascending=[True, False])
    .reset_index(drop=True)
)
        
topk_keep_seen_with_titles.head(10)

,user_id,item_id,score,item_id_name
0,1,50,16763,Star Wars (1977)
1,1,181,15297,Return of the Jedi (1983)
2,1,174,15081,Raiders of the Lost Ark (1981)
3,1,1,14654,Toy Story (1995)
4,1,100,13913,Fargo (1996)
5,1,98,13800,"Silence of the Lambs, The (1991)"
6,1,172,13776,"Empire Strikes Back, The (1980)"
7,1,210,13513,Indiana Jones and the Last Crusade (1989)
8,1,121,12923,Independence Day (ID4) (1996)
9,1,222,12897,Star Trek: First Contact (1996)


### 2.3. Evaluate how well FBT performs

We evaluate how well FBT performs for a few common ranking metrics provided in the `python_evaluation` module in reco_utils. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with FBT. User and item column names are specified in each evaluation method. Since FBT does not have ratings information, we create a dummy column with all values set to 1.0 so as to conform to the metrics signature.

In [24]:
test['rating'] = 1
eval_map_k = map_at_k(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score,k=TOP_K)
print(f"MAP@{TOP_K}: {eval_map_k}")

MAP@10: 0.0700594112753097


In [25]:
eval_ndcg = ndcg_at_k(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score, k=TOP_K)
print(f"NDCG@{TOP_K}: {eval_ndcg}")

NDCG@10: 0.30469469553920064


In [26]:
eval_precision = precision_at_k(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score, k=TOP_K)
print(f"Precision@{TOP_K}: {eval_precision}")

Precision@10: 0.26521739130434785


In [27]:
eval_recall = recall_at_k(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score, k=TOP_K)
print(f"Recall@{TOP_K}: {eval_recall}")

Recall@10: 0.1250000517480203


In [28]:
eval_rmse = rmse(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score)
print(f"RMSE: {eval_rmse}")

RMSE: 8901.019858835787


In [29]:
eval_mae = mae(test, topk_remove_seen, col_user=col_user, col_item=col_item, col_prediction=col_score)
print(f"MAE: {eval_mae}")

MAE: 7699.956417433027


In [30]:
print(f"Model:\t",
      f"Top K: {TOP_K}",
      f"MAP@{TOP_K}: {eval_map_k}",
      f"NDCG@{TOP_K}: {eval_ndcg}",
      f"Precision@{TOP_K}: {eval_precision}",
      f"Recall@{TOP_K}: {eval_recall}",
      f"RMSE: {eval_rmse}",
      f"MAE: {eval_mae}",
      sep='\n')

Model:	
Top K: 10
MAP@10: 0.0700594112753097
NDCG@10: 0.30469469553920064
Precision@10: 0.26521739130434785
Recall@10: 0.1250000517480203
RMSE: 8901.019858835787
MAE: 7699.956417433027


In [32]:
# Now let's look at the results for a specific user
user_id = 1

ground_truth = test[test[col_user]==user_id]
prediction = topk_remove_seen[topk_remove_seen[col_user]==user_id].sort_values(by=col_score, ascending=False)[:TOP_K]
test_user_movie_watched_prediction = (
    pd.merge(ground_truth, prediction, on=[col_user, col_item], how='left')
      .drop(columns=['rating'])
)
test_user_movie_watched_prediction.head()

,user_id,item_id,item_id_name,score
0,1,49,I.Q. (1994),NaN
1,1,69,Forrest Gump (1994),12159.0
2,1,221,Breaking the Waves (1996),NaN
3,1,5,Copycat (1995),NaN
4,1,139,"Love Bug, The (1969)",NaN


Above, we see that one of the movies from the test set was recovered by the model's top-k recommendations, however the others were not. Offline evaluations are difficult as they can only use what was seen previously in the test set and may not represent the user's actual preferences across the entire set of items. Adjustments to how the data is split, algorithm is used and hyper-parameters can improve the results here. 

In [44]:
# Record results with papermill for tests - ignore this cell
sb.glue("map", eval_map_k)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)
sb.glue("train_time", train_time.interval)
sb.glue("test_time", test_time.interval)